In [24]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
movies = pd.read_csv('Data/movies.csv')
ratings = pd.read_csv('Data/ratings.csv')

In [51]:
merged_data = pd.merge(ratings, movies, on='movieId')
merged_data.drop(columns=['timestamp'], inplace=True)
merged_data[merged_data['movieId'] == 1].head(50)

,userId,movieId,rating,title,genres
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
5,18,1,3.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
6,19,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
7,21,1,3.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
8,27,1,3.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
9,31,1,5.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [19]:
print("Number of unique users:", merged_data['userId'].nunique())
print("Number of unique movies:", merged_data['movieId'].nunique())
print("Number of ratings:", merged_data['rating'].count())

Number of unique users: 610
Number of unique movies: 9724
Number of ratings: 100836


In [18]:
print(merged_data.isnull().sum().sum())
print(merged_data.duplicated().sum())

0
0


In [23]:
data_matrix = merged_data.pivot_table(index='userId', columns='movieId', values='rating')
data_matrix.fillna(0, inplace=True)
data_matrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
cs = cosine_similarity(data_matrix)
cs_df = pd.DataFrame(cs, index=data_matrix.index, columns=data_matrix.index)
cs_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.027283,0.059720,0.194395,0.129080,0.128152,0.158744,0.136968,0.064263,0.016875,...,0.080554,0.164455,0.221486,0.070669,0.153625,0.164191,0.269389,0.291097,0.093572,0.145321
2,0.027283,1.000000,0.000000,0.003726,0.016614,0.025333,0.027585,0.027257,0.000000,0.067445,...,0.202671,0.016866,0.011997,0.000000,0.000000,0.028429,0.012948,0.046211,0.027565,0.102427
3,0.059720,0.000000,1.000000,0.002251,0.005020,0.003936,0.000000,0.004941,0.000000,0.000000,...,0.005048,0.004892,0.024992,0.000000,0.010694,0.012993,0.019247,0.021128,0.000000,0.032119
4,0.194395,0.003726,0.002251,1.000000,0.128659,0.088491,0.115120,0.062969,0.011361,0.031163,...,0.085938,0.128273,0.307973,0.052985,0.084584,0.200395,0.131746,0.149858,0.032198,0.107683
5,0.129080,0.016614,0.005020,0.128659,1.000000,0.300349,0.108342,0.429075,0.000000,0.030611,...,0.068048,0.418747,0.110148,0.258773,0.148758,0.106435,0.152866,0.135535,0.261232,0.060792


In [56]:
def generate_recommendation(userId, SimilarUsers = 5, TopMovies = 10):
  user_similarities = cs_df[userId].sort_values(ascending=False)
  top_users = user_similarities[1:SimilarUsers+1]
  top_movies = []
  for user in top_users.index:
      user_movies = data_matrix.loc[user].sort_values(ascending=False)
      user_movies = user_movies[user_movies > 0].head()
      top_movies.extend(user_movies[user_movies > 0].index.tolist())
  print(top_movies)
  top_movies = pd.Series(top_movies).value_counts().head(TopMovies)
  recommended_movies = movies[movies['movieId'].isin(top_movies.index)]
  print(f"Recommendations for User {userId}:")
  print(recommended_movies[['title']])


generate_recommendation(1)

[1136, 1261, 1196, 2700, 2599, 2702, 1394, 2947, 1198, 2195, 2194, 1199, 1200, 1208, 1089, 1663, 1884, 110, 514, 1266, 2968, 2916, 2571, 2959, 2890, 2712, 2278, 2716, 69, 1732, 608, 1610, 1220, 924, 1285, 555, 1466, 16, 353, 1288, 296, 3108, 1206, 2683, 1090, 2064, 1222, 1213, 3060, 1298, 3052, 1297, 1291, 1079, 778, 733, 2300, 3265, 541, 1080, 2167, 2985, 2115, 2387, 1573, 1242, 2863, 1093, 1240, 2288, 3039, 785, 1036, 356, 1617, 223, 50, 2804, 1127, 2560, 6, 480, 1270, 1197, 1265, 260, 1275, 2539, 1476, 589, 21, 592, 457, 2797, 32, 1259, 1912, 2580, 235, 2028, 1517, 2105, 2174, 2355, 1641, 2353, 1527, 1042, 1923, 3016, 1784, 493, 1374, 3020, 1171, 2770, 2812, 1375, 145, 316, 1210, 1580, 1500, 1356, 198, 2000, 367, 474, 95, 441, 1215, 2094, 1722, 2395, 1485, 2097, 551, 1405, 1097, 2321, 1597, 1060, 1866, 132, 2249, 440, 1, 39, 2490, 1968, 377, 2918, 45, 2471, 1911, 2605, 2384, 694, 3156, 1883, 64, 2253, 476, 3142, 1772, 2791, 3113, 24, 919, 17, 288, 2710, 2302, 2396, 2706, 1614, 135, 